Loading image dataset and CSV Files

In [ ]:
import os 
import pandas
import numpy 
from augmentations import augmentations
from datasets import datasets
from PIL import Image
import cv2

images = []

for fil in os.listdir("../data/raw_data/images"):
    file_dir = os.path.join("../data/raw/data/images", fil)
    images.append(file_dir)

Loading Dataset information CSV File

In [ ]:
image_loader = numpy.vectorize(pyfunc=lambda img: Image.open(img))

train_info = pandas.read_csv("../data/raw_data/information.csv")
train_info['image'] = image_loader(images)

In [ ]:
train_info['Class'].map({
    'yes': 0,
    'no': 1,
})

Converting images to specific format

In [ ]:
def to_png(img):
    success, png_data = cv2.imencode(ext='.png', img=img)
    if success:
        png_img = cv2.imdecode(png_data, cv2.IMREAD_UNCHANGED)
        return Image.fromarray(png_img)
    else:
        raise RuntimeError('Failed to convert image')

train_info['image'] = train_info['image'].apply(lambda img: to_png(img))

Checking data for Gaussian Noise, Impulse Noise & Salt-And-Papper Noise

In [ ]:
def has_gaussian_noise(img, threshold: float):
    histogram = cv2.calcHist(
        images=[img], 
        channels=[0],
        mask=None, 
        ranges=[0, 256]
    )  
    return numpy.std(histogram) > threshold

def has_salt_and_paper_noise(img, threshold: float):
    low_d = numpy.count_nonzero(numpy.where(img == 0, 1, 0))
    high_d = numpy.count_nonzero(numpy.where(img == 255, 1, 0))
    return (low_d + high_d) / img.shape[0] > threshold

In [ ]:
images = train_info['image'].tolist()

gaussian_thresh = 10 
impulse_thresh = 0.8
sp_thresh = 0.8

sp_images = []
gaussian_images = []

for idx, image in enumerate(images):

    if has_gaussian_noise(image, gaussian_thresh):
        gaussian_images.append((idx, image))

    if has_salt_and_paper_noise(image, sp_thresh):
        sp_images.append((idx, image))

Cleaning image fragments

In [ ]:
kernel_size = 10
color_range = 20 
space_range = 50

for idx, img in [*gaussian_images, *sp_images]:
    images[idx] = cv2.bilateralFilter(
        src=img,
        d=kernel_size,
        sigmaColor=color_range,
        sigma_space=space_range,
    )

Initializing Datasets for training and validation

In [ ]:
from sklearn.model_selection import train_test_split 

train_indices, val_indices = train_test_split(
    numpy.arrange(train_info.shape[0]), 
    test_size=0.3, 
    stratify=train_info['class'].tolist()
)

In [ ]:
train_dataset = datasets.DeepFakeClassificationDataset(
    labels=train_info['class'][train_indices],
    images=train_info['image'][train_indices],
    transforms=augmentations.get_training_augmentations()
)

validation_dataset = datasets.DeepFakeClassificationDataset(
    labels=train_info['class'][val_indices],
    images=train_info['image'][val_indices],
    transforms=augmentations.get_validation_augmentations()
)

Handling disbalance in the dataset using Class Weighting

In [ ]:
CLASS_WEIGHTS = [1, 1]

train_dataset.weights = validation_dataset.weights = CLASS_WEIGHTS

Saving datasets to local storage

In [ ]:
import pickle 

pickle.dump(obj=train_dataset, file=open("../data/augmented_data/train_dataset.pkl", mode='wb'))
pickle.dump(obj=validation_dataset, file=open("../data/augmented_data/validation_dataset.pkl", mode='wb'))